In [1]:
import pandas as pd
from input_data import input_df
import scipy.stats
import matplotlib.pyplot as plt
from pre2 import data_pre
from m01_lightgbm import train

In [72]:
from sklearn.cluster import KMeans
def data_pre2(df, ibaragi_params):
    
    df = data_pre(df)
    
    area_df = pd.read_csv("./feature_data/axis_cluster.csv")
    
    # 茨城県北相馬郡利根町はtrainにない(クラスタ平均:1.241732)
    ibaragi = pd.DataFrame([ibaragi_params], columns=["クラスタ平均"])
    ibaragi["市区町村名"] = "北相馬郡利根町"
    area_df = pd.concat([area_df, ibaragi], axis=0)
    
    df = pd.merge(df.reset_index(), area_df, on="市区町村名").set_index("ID")
    
    
    for col in ["都道府県名", "市区町村名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
        df[col] = df[col].astype("category")
        
    df = df.drop("都道府県名_en", axis=1)
    
    
    return df

In [38]:
df = input_df()
df = data_pre(df)
df_b = df.copy()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [39]:
df = df[["都道府県名", "市区町村名", "緯度", "経度", "面積（㎡）", "取引価格（総額）_log"]]

In [40]:
pref_list = df["都道府県名"].tolist()
pref_list = set(pref_list)

In [41]:
df_z = pd.DataFrame()

for pref in pref_list:
    pref_df = df[df["都道府県名"]==pref]["取引価格（総額）_log"]
    zscore_list = pref_df.tolist()
    df_t = df[df["都道府県名"]==pref]
    df_t["zscore"] = scipy.stats.zscore(zscore_list)
    df_t = df_t[["zscore"]]
    df_z = pd.concat([df_z, df_t], axis=0)

df = pd.merge(df, df_z, on="ID")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [42]:
from sklearn.cluster import KMeans
pred = KMeans(n_clusters=4).fit_predict(df[["緯度", "経度", "zscore"]])

In [43]:
df["クラスタ"] = pred

In [44]:
df.head()

,都道府県名,市区町村名,緯度,経度,面積（㎡）,取引価格（総額）_log,zscore,クラスタ
ID,,,,,,,,
1117225,北海道,苫小牧市,42.631246,141.603122,120.0,7.431364,1.231246,3
1114724,北海道,苫小牧市,42.631246,141.603122,70.0,6.342423,-1.764705,3
1114753,北海道,苫小牧市,42.631246,141.603122,70.0,6.778151,-0.565906,3
1120806,北海道,苫小牧市,42.631246,141.603122,105.0,7.230449,0.678479,3
1117264,北海道,苫小牧市,42.631246,141.603122,90.0,7.204120,0.606041,3


In [45]:
df = df[["zscore", "クラスタ"]]

In [46]:
df = pd.merge(df_b, df, on="ID")

In [47]:
df.tail()

,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,...,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,緯度,経度,都道府県名_en,zscore,クラスタ
ID,,,,,,,,,,,,,,,,,,,,,
47008534,沖縄県,うるま市,字宮里,NaN,NaN,４ＬＤＫ,65.0,10.0,ＳＲＣ,住宅,...,200.0,2019.75,未改装,NaN,7.342423,26.378685,127.857493,47,0.452388,2
47007866,沖縄県,うるま市,字江洲,NaN,NaN,３ＬＤＫ,70.0,6.0,ＲＣ,住宅,...,200.0,2018.99,改装済,NaN,7.447158,26.378685,127.857493,47,0.788960,2
47029524,沖縄県,うるま市,字塩屋,NaN,NaN,３ＬＤＫ,75.0,22.0,ＲＣ,住宅,...,100.0,2009.99,未改装,NaN,7.000000,26.378685,127.857493,47,-0.648002,2
47008285,沖縄県,うるま市,字塩屋,NaN,NaN,３ＬＤＫ,75.0,22.0,ＲＣ,住宅,...,100.0,2015.25,未改装,NaN,7.113943,26.378685,127.857493,47,-0.281840,2
47008266,沖縄県,うるま市,字塩屋,NaN,NaN,３ＬＤＫ,75.0,22.0,ＲＣ,住宅,...,NaN,2019.75,改装済,NaN,7.176091,26.378685,127.857493,47,-0.082125,2


In [48]:
area_list = set(df["市区町村名"].tolist())

In [49]:
area_class_mean = []
for area in area_list:
    area_mean = df[df["市区町村名"] == area]["クラスタ"].mean()
    area_class_mean.append(area_mean)

df_area = pd.DataFrame(area_class_mean, columns = ['クラスタ平均'])
df_area["市区町村名"] = area_list

In [50]:
df

,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,...,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,緯度,経度,都道府県名_en,zscore,クラスタ
ID,,,,,,,,,,,,,,,,,,,,,
1117225,北海道,苫小牧市,新中野町,苫小牧,25.0,３ＬＤＫ,120.0,12.0,ＲＣ,住宅,...,200.0,2018.75,未改装,NaN,7.431364,42.631246,141.603122,1,1.231246,3
1114724,北海道,苫小牧市,青葉町,青葉,9.0,３ＬＤＫ,70.0,35.0,ＲＣ,住宅,...,200.0,2017.75,未改装,NaN,6.342423,42.631246,141.603122,1,-1.764705,3
1114753,北海道,苫小牧市,青葉町,青葉,3.0,３ＬＤＫ,70.0,31.0,ＲＣ,住宅,...,200.0,2016.50,改装済,NaN,6.778151,42.631246,141.603122,1,-0.565906,3
1120806,北海道,苫小牧市,表町,苫小牧,8.0,４ＬＤＫ,105.0,18.0,ＲＣ,住宅,...,400.0,2011.99,未改装,NaN,7.230449,42.631246,141.603122,1,0.678479,3
1117264,北海道,苫小牧市,新中野町,苫小牧,25.0,３ＬＤＫ,90.0,15.0,ＲＣ,住宅,...,200.0,2015.50,未改装,NaN,7.204120,42.631246,141.603122,1,0.606041,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47008534,沖縄県,うるま市,字宮里,NaN,NaN,４ＬＤＫ,65.0,10.0,ＳＲＣ,住宅,...,200.0,2019.75,未改装,NaN,7.342423,26.378685,127.857493,47,0.452388,2
47007866,沖縄県,うるま市,字江洲,NaN,NaN,３ＬＤＫ,70.0,6.0,ＲＣ,住宅,...,200.0,2018.99,改装済,NaN,7.447158,26.378685,127.857493,47,0.788960,2
47029524,沖縄県,うるま市,字塩屋,NaN,NaN,３ＬＤＫ,75.0,22.0,ＲＣ,住宅,...,100.0,2009.99,未改装,NaN,7.000000,26.378685,127.857493,47,-0.648002,2


In [51]:
df_area

,クラスタ平均,市区町村名
0,1.0,大阪市東住吉区
1,3.0,札幌市西区
2,0.0,塩竈市
3,0.0,茅ヶ崎市
4,1.0,高浜市
...,...,...
613,1.0,岡山市北区
614,1.0,知多郡南知多町
615,0.0,昭島市
616,0.0,熊谷市


In [1]:
df_area.to_csv('./feature_data/axis_cluster.csv', index=False)

with open('./feature_data/axis_cluster.csv', 'a') as f:
    print('0.0,北相馬郡利根町', file=f)

In [53]:
df = pd.merge(df, df_area, on="市区町村名")
df["市区町村名"] = df["市区町村名"].astype("category")

In [54]:
df = df.drop("都道府県名_en", axis=1)

In [55]:
df.head()

,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,...,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,緯度,経度,zscore,クラスタ,クラスタ平均
0,北海道,苫小牧市,新中野町,苫小牧,25.0,３ＬＤＫ,120.0,12.0,ＲＣ,住宅,...,200.0,2018.75,未改装,NaN,7.431364,42.631246,141.603122,1.231246,3,3.0
1,北海道,苫小牧市,青葉町,青葉,9.0,３ＬＤＫ,70.0,35.0,ＲＣ,住宅,...,200.0,2017.75,未改装,NaN,6.342423,42.631246,141.603122,-1.764705,3,3.0
2,北海道,苫小牧市,青葉町,青葉,3.0,３ＬＤＫ,70.0,31.0,ＲＣ,住宅,...,200.0,2016.50,改装済,NaN,6.778151,42.631246,141.603122,-0.565906,3,3.0
3,北海道,苫小牧市,表町,苫小牧,8.0,４ＬＤＫ,105.0,18.0,ＲＣ,住宅,...,400.0,2011.99,未改装,NaN,7.230449,42.631246,141.603122,0.678479,3,3.0
4,北海道,苫小牧市,新中野町,苫小牧,25.0,３ＬＤＫ,90.0,15.0,ＲＣ,住宅,...,200.0,2015.50,未改装,NaN,7.204120,42.631246,141.603122,0.606041,3,3.0


In [56]:
df = df.drop(['zscore', 'クラスタ'], axis=1)

In [57]:
df.head()

,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,...,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log,緯度,経度,クラスタ平均
0,北海道,苫小牧市,新中野町,苫小牧,25.0,３ＬＤＫ,120.0,12.0,ＲＣ,住宅,...,準工業地域,60.0,200.0,2018.75,未改装,NaN,7.431364,42.631246,141.603122,3.0
1,北海道,苫小牧市,青葉町,青葉,9.0,３ＬＤＫ,70.0,35.0,ＲＣ,住宅,...,第１種中高層住居専用地域,60.0,200.0,2017.75,未改装,NaN,6.342423,42.631246,141.603122,3.0
2,北海道,苫小牧市,青葉町,青葉,3.0,３ＬＤＫ,70.0,31.0,ＲＣ,住宅,...,第１種中高層住居専用地域,60.0,200.0,2016.50,改装済,NaN,6.778151,42.631246,141.603122,3.0
3,北海道,苫小牧市,表町,苫小牧,8.0,４ＬＤＫ,105.0,18.0,ＲＣ,住宅,...,商業地域,80.0,400.0,2011.99,未改装,NaN,7.230449,42.631246,141.603122,3.0
4,北海道,苫小牧市,新中野町,苫小牧,25.0,３ＬＤＫ,90.0,15.0,ＲＣ,住宅,...,準工業地域,60.0,200.0,2015.50,未改装,NaN,7.204120,42.631246,141.603122,3.0


In [58]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
import lightgbm
import optuna.integration.lightgbm as lgb


df_train, df_val = train_test_split(df, test_size=0.2)

col = "取引価格（総額）_log"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

In [67]:
opt_params = {
    "objective":"regression",
    "metric":"mae"
}

import time
start = time.time()
opt=lgb.train(
    opt_params,
    trains,
    valid_sets = valids,
    verbose_eval=200,
    num_boost_round = 1000,
    early_stopping_rounds = 100,
    time_budget = 600
)
end = time.time()
print('elaptedtime:', end-start)

[I 2021-05-28 13:50:57,959] A new study created in memory with name: no-name-7f5f539a-e4c6-4798-93a0-d5fe1956eddc



  0%|          | 0/7 [00:00<?, ?it/s]


feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.035662 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12338
[LightGBM] [Info] Number of data points in the train set: 521580, number of used features: 20
[LightGBM] [Info] Start training from score 7.219047
Training until validation scores don't improve for 100 rounds
[200]	valid_0's l1: 0.0778997
[400]	valid_0's l1: 0.0761981
[600]	valid_0's l1: 0.0756095
[800]	valid_0's l1: 0.0753015
[1000]	valid_0's l1: 0.0750958
Did not meet early stopping. Best iteration is:
[998]	valid_0's l1: 0.0750954





feature_fraction, val_score: 0.075095:   0%|          | 0/7 [00:23<?, ?it/s]


feature_fraction, val_score: 0.075095:  14%|#4        | 1/7 [00:23<02:19, 23.18s/it][I 2021-05-28 13:51:21,143] Trial 0 finished with value: 0.0750954162473413 and parameters: {'feature_fraction': 0.6}. Best is trial 0 with value: 0.0750954162473413.



feature_fraction, val_score: 0.075095:  14%|#4        | 1/7 [00:23<02:19, 23.18s/it]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.034834 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12338
[LightGBM] [Info] Number of data points in the train set: 521580, number of used features: 20
[LightGBM] [Info] Start training from score 7.219047
Training until validation scores don't improve for 100 rounds
[200]	valid_0's l1: 0.0777327
[400]	valid_0's l1: 0.0762166
[600]	valid_0's l1: 0.0757582
[800]	valid_0's l1: 0.0754934
[1000]	valid_0's l1: 0.0754026
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l1: 0.0754026





feature_fraction, val_score: 0.075095:  14%|#4        | 1/7 [00:49<02:19, 23.18s/it]


feature_fraction, val_score: 0.075095:  29%|##8       | 2/7 [00:49<02:04, 24.99s/it][I 2021-05-28 13:51:47,399] Trial 1 finished with value: 0.07540260675263794 and parameters: {'feature_fraction': 0.8}. Best is trial 0 with value: 0.0750954162473413.



feature_fraction, val_score: 0.075095:  29%|##8       | 2/7 [00:49<02:04, 24.99s/it]

...

[1000]	valid_0's l1: 0.0771297
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l1: 0.0771297
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009113 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12338
[LightGBM] [Info] Number of data points in the train set: 521580, number of used features: 20
[LightGBM] [Info] Start training from score 7.219047
Training until validation scores don't improve for 100 rounds
[200]	valid_0's l1: 0.075505
[400]	valid_0's l1: 0.0747599
[600]	valid_0's l1: 0.0746444
[800]	valid_0's l1: 0.0745608
[1000]	valid_0's l1: 0.0745079
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l1: 0.0745079





num_leaves, val_score: 0.074445:  65%|######5   | 13/20 [06:32<02:51, 24.54s/it]


num_leaves, val_score: 0.074445:  70%|#######   | 14/20 [06:32<02:45, 27.56s/it][I 2021-05-28 14:00:26,584] Trial 20 finished with value: 0.07450789929537288 and parameters: {'num_leaves': 111}. Best is trial 11 with value: 0.07444525775506969.



num_leaves, val_score: 0.074445:  70%|#######   | 14/20 [06:32<02:45, 27.56s/it]

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009705 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12338
[LightGBM] [Info] Number of data points in the train set: 521580, number of used features: 20
[LightGBM] [Info] Start training from score 7.219047
Training until validation scores don't improve for 100 rounds
[200]	valid_0's l1: 0.075116
[400]	valid_0's l1: 0.0746805
[600]	valid_0's l1: 0.0746552
Early stopping, best iteration is:
[526]	valid_0's l1: 0.0746393





num_leaves, val_score: 0.074445:  70%|#######   | 14/20 [07:04<02:45, 27.56s/it]


num_leaves, val_score: 0.074445:  75%|#######5  | 15/20 [07:04<02:21, 28.33s/it]



  0%|          | 0/10 [00:00<?, ?it/s]



  0%|          | 0/3 [00:00<?, ?it/s]



  0%|          | 0/20 [00:00<?, ?it/s]



  0%|          | 0/5 [00:00<?, ?it/s]

elaptedtime: 601.1325197219849


In [68]:
best_params = opt.params
print(best_params)

{'objective': 'regression', 'metric': 'l1', 'feature_pre_filter': False, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 71, 'feature_fraction': 0.4, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 100, 'categorical_column': [0, 1, 2, 3, 5, 8, 9, 10, 11, 15, 16]}


In [69]:
# lightGBM
import lightgbm as lgb

# params = {
#     "objective": "regression",
#     "metrics": "mae", 
#     "lambda_l1": 8.442195346804022,
#     "lambda_l2": 5.708301125509865e-05,
#     "num_leaves": 143,
#     "feature_fraction": 0.484,
#     "bagging_fraction": 1.0,
#     "bagging_freq": 0,
#     "min_child_samples": 50
# }


model = lgb.train(best_params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100, verbose_eval=200)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009242 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12338
[LightGBM] [Info] Number of data points in the train set: 521580, number of used features: 20
[LightGBM] [Info] Start training from score 7.219047
Training until validation scores don't improve for 100 rounds
[200]	valid_0's l1: 0.0761363
[400]	valid_0's l1: 0.0750252
[600]	valid_0's l1: 0.074717
[800]	valid_0's l1: 0.0745456
[1000]	valid_0's l1: 0.0744465
Did not meet early stopping. Best iteration is:
[998]	valid_0's l1: 0.0744453


In [70]:
vals = model.predict(val_x)
mae(vals, val_y)

0.07444525911426259

In [30]:
# import pickle
# file = '01_trained_model.pkl'
# pickle.dump(model, open(file, 'wb'))

In [73]:
ibaragi_params = df[df["都道府県名"]=="茨城県"]["クラスタ平均"].mean()

In [74]:
df_test = pd.read_csv("/tmp/working/dataset/nishika/old_apartment_2020/test.csv", index_col=0)
df_test = data_pre2(df_test, ibaragi_params)
predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict
df_test[["取引価格（総額）_log"]].to_csv("./output/08_sub.csv")

In [76]:
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
地区名,9553
最寄駅：名称,8511
取引時点,8107
建築年,6800
市区町村名,5813
最寄駅：距離（分）,5689
面積（㎡）,5061
緯度,3321
経度,2960
間取り,2549
